In [1]:
import pandas as pd
import numpy as np

In [5]:
# 1. Đọc dữ liệu giá đóng cửa quý (CSV) và chuẩn hoá tên ticker
bank_df = pd.read_csv(
    r'C:\code\R\HK3-2025\doangiuaky\python crawing data\bank_stocks_monthly_close.csv',
    parse_dates=['Tháng'],
    encoding='cp1258'
)
bank_df = bank_df.set_index('Tháng').sort_index()

In [7]:
# 2. Đọc dữ liệu số lượng cổ phiếu đang lưu hành (Excel), skip dòng đầu không chứa dữ liệu
shares_df = pd.read_excel(
    'C:\code\R\HK3-2025\doangiuaky\python crawing data\Shares_outstanding.xlsx',
    skiprows=1,  # Bỏ dòng mô tả "Common Shares - Outstanding - Total"
    header=None,
    names=['ticker_full', 'date', 'shares_raw']
)

In [8]:
# 3. Chuyển cột 'date' sang datetime (dayfirst=True) và làm sạch cột shares_raw
shares_df['date'] = pd.to_datetime(shares_df['date'], dayfirst=True, errors='coerce')
shares_df['shares'] = (
    shares_df['shares_raw']
    .astype(str)
    .str.replace(r'[\,\.]', '', regex=True)  # loại bỏ mọi dấu ',' và '.'
    .astype(float)
)

In [9]:
# 4. Chỉ giữ lại các cột cần thiết: ticker_full, date, shares
shares_df = shares_df[['ticker_full', 'date', 'shares']].dropna(subset=['date', 'shares'])
shares_df = shares_df.sort_values(['ticker_full', 'date']).reset_index(drop=True)

In [10]:
# 5. In 20 dòng đầu của shares_df để kiểm tra
print("=== 20 dòng đầu của shares_df ===")
print(shares_df.head(20))

# Xuất toàn bộ bank_long ra file Excel
shares_df.to_excel('share_df_full.xlsx', index=False)
print("Đã xuất toàn bộ share_df_full vào 'bank_long_full.xlsx'")


=== 20 dòng đầu của shares_df ===
   ticker_full       date        shares
0       ACB.HM 2004-12-31  1.897461e+14
1       ACB.HM 2005-12-31  1.897461e+14
2       ACB.HM 2006-12-31  1.897441e+14
3       ACB.HM 2007-12-31  3.489629e+14
4       ACB.HM 2008-12-31  3.981914e+14
5       ACB.HM 2009-12-31  4.827468e+13
6       ACB.HM 2010-12-31  5.240080e+14
7       ACB.HM 2011-12-31  5.240080e+14
8       ACB.HM 2012-12-31  5.240080e+14
9       ACB.HM 2013-12-31  5.149656e+14
10      ACB.HM 2014-12-31  5.008818e+14
11      ACB.HM 2015-12-31  5.008601e+14
12      ACB.HM 2016-12-31  5.008601e+14
13      ACB.HM 2017-12-31  5.008596e+14
14      ACB.HM 2018-12-31  5.008618e+14
15      ACB.HM 2019-12-31  5.117356e+13
16      ACB.HM 2020-12-31  5.136572e+14
17      ACB.HM 2021-12-31  5.136572e+14
18      ACB.HM 2022-12-31  5.136572e+14
19      ACB.HM 2023-12-31  5.136600e+14
Đã xuất toàn bộ share_df_full vào 'bank_long_full.xlsx'


In [12]:
# 6. Đưa bảng giá quý về dạng long và tạo cột ticker_full phù hợp
hn_tickers = {'NVB', 'BAB'}

bank_long = bank_df.reset_index().melt(
    id_vars=['Tháng'],
    var_name='ticker',
    value_name='price'
).rename(columns={'Tháng': 'date'})

def make_full_ticker(ticker: str) -> str:
    return f"{ticker}.HN" if ticker in hn_tickers else f"{ticker}.HM"

bank_long['ticker_full'] = bank_long['ticker'].apply(make_full_ticker)
bank_long = bank_long[['date', 'ticker_full', 'price']]
bank_long = bank_long.sort_values(['ticker_full', 'date']).reset_index(drop=True)

In [13]:
# 7. In 20 dòng đầu của bank_long để kiểm tra
print("\n=== 20 dòng đầu của bank_long (giá đóng cửa) ===")
print(bank_long.head(20))

# Xuất toàn bộ bank_long ra file Excel
bank_long.to_excel('bank_long_full.xlsx', index=False)
print("Đã xuất toàn bộ bank_long vào 'bank_long_full.xlsx'")


=== 20 dòng đầu của bank_long (giá đóng cửa) ===
         date ticker_full  price
0  2010-01-31      ACB.HM   3.52
1  2010-02-28      ACB.HM   3.66
2  2010-03-31      ACB.HM   3.58
3  2010-04-30      ACB.HM   3.53
4  2010-05-31      ACB.HM   3.29
5  2010-06-30      ACB.HM   3.19
6  2010-07-31      ACB.HM   3.13
7  2010-08-31      ACB.HM   2.95
8  2010-09-30      ACB.HM   2.98
9  2010-10-31      ACB.HM   2.81
10 2010-11-30      ACB.HM   2.76
11 2010-12-31      ACB.HM   3.21
12 2011-01-31      ACB.HM   2.99
13 2011-02-28      ACB.HM   2.76
14 2011-03-31      ACB.HM   2.82
15 2011-04-30      ACB.HM   2.76
16 2011-05-31      ACB.HM   2.62
17 2011-06-30      ACB.HM   2.65
18 2011-07-31      ACB.HM   2.67
19 2011-08-31      ACB.HM   2.71
Đã xuất toàn bộ bank_long vào 'bank_long_full.xlsx'


In [14]:
# 8. Merge_asof theo từng ticker_full để lấy shares gần nhất ≤ ngày đó
merged_list = []
for ticker, grp_bank in bank_long.groupby('ticker_full'):
    grp_bank = grp_bank.sort_values('date').reset_index(drop=True)
    grp_shares = shares_df[shares_df['ticker_full'] == ticker][['date', 'shares']].sort_values('date').reset_index(drop=True)
    if grp_shares.empty:
        merged_list.append(grp_bank.assign(shares=pd.NA))
    else:
        merged_grp = pd.merge_asof(
            grp_bank,
            grp_shares,
            on='date',
            direction='backward'
        )
        merged_grp['ticker_full'] = ticker
        merged_list.append(merged_grp)

merged = pd.concat(merged_list, ignore_index=True)

In [15]:
# 9. Tính market cap = price * shares
merged['market_cap'] = merged['price'] * merged['shares']

In [16]:
# 10. Pivot thành wide: mỗi hàng là một ngày (quý), cột là ticker_full
mcap_wide = merged.pivot_table(
    index='date',
    columns='ticker_full',
    values='market_cap'
).sort_index()

In [17]:
# 11. Tính TotalCap mỗi quý
total_mcap = mcap_wide.sum(axis=1, skipna=True)

In [18]:
# 12. Khởi tạo divisor và index với base = 100
dates = total_mcap.index.to_list()
t0 = dates[0]
base_index_value = 100.0
total_t0 = total_mcap.loc[t0]

divisors = pd.Series(index=dates, dtype='float64')
divisors[t0] = total_t0 / base_index_value

In [19]:
# 13. Tính divisor cho các quý sau, điều chỉnh khi có thêm/bớt ticker
for i in range(1, len(dates)):
    today = dates[i]
    yesterday = dates[i - 1]

    total_prev = total_mcap.loc[yesterday]
    total_curr = total_mcap.loc[today]

    prev_active = mcap_wide.loc[yesterday].notna()
    curr_active = mcap_wide.loc[today].notna()

    added = curr_active & (~prev_active)
    removed = (~curr_active) & prev_active

    if added.any() or removed.any():
        divisors[today] = divisors[yesterday] * (total_curr / total_prev)
    else:
        divisors[today] = divisors[yesterday]


In [20]:
# 14. Tính Bank Index = TotalCap / Divisor
bank_index = total_mcap / divisors

In [21]:
# 15. Tạo DataFrame kết quả với cột thời gian ở đầu
result = pd.DataFrame({
    'date': total_mcap.index,
    'Total_Market_Cap': total_mcap.values,
    'Divisor': divisors.values,
    'Bank_Index': bank_index.values
})

In [22]:
# 16. Tính log‐return của chỉ số ngân hàng (Bank_return)
result['Bank_return'] = np.log(result['Bank_Index'] / result['Bank_Index'].shift(1))

In [ ]:
print("\n=== 10 dòng đầu của kết quả Bank Index (base=100) ===")
print(result.head(10))

# Xuất toàn bộ bank_long ra file Excel
result.to_excel('banking_return.xlsx', index=False)
print("Đã xuất toàn bộ result vào 'banking_return.xlsx'")



=== 10 dòng đầu của kết quả Bank Index (base=100) ===
        date  Total_Market_Cap       Divisor  Bank_Index  Bank_return
0 2010-01-31      5.860208e+15  5.860208e+13  100.000000          NaN
1 2010-02-28      6.623763e+15  5.860208e+13  113.029488     0.122479
2 2010-03-31      6.030124e+15  5.860208e+13  102.899484    -0.093896
3 2010-04-30      5.971474e+15  5.860208e+13  101.898671    -0.009774
4 2010-05-31      5.663621e+15  5.860208e+13   96.645398    -0.052930
5 2010-06-30      5.843799e+15  5.860208e+13   99.719989     0.031318
6 2010-07-31      5.502347e+15  5.860208e+13   93.893370    -0.060206
7 2010-08-31      5.325888e+15  5.860208e+13   90.882244    -0.032595
8 2010-09-30      5.339513e+15  5.875200e+13   90.882244     0.000000
9 2010-10-31      5.188472e+15  5.875200e+13   88.311407    -0.028695
Đã xuất toàn bộ result vào 'bank_long_full.xlsx'
